In [1]:
!pip install folium
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
import folium

You should consider upgrading via the 'c:\users\munna.iv\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


Import data

In [2]:
CLIENT_ID = '1W5TLVXZFWOJO3ZYPQQLHYAH3EOPKOSKJJEUDXQCZKZDUVW1' 
CLIENT_SECRET = 'ZIGTLKPAKXIKDRXF3BPTCMJ24QI01I502FBMBUGCYSCTTNFF' 
VERSION = '20210118'

In [3]:
LIMIT = 500 
cities = ['Los Angeles', 'New York', 'London', 'Milan', 'Barcelona']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        '4d4b7105d754a06376d81259') 
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

Create a map to display the nightlife spots in each city

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.Marker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of nightlife spots {city} = ", results[city]['response']['totalResults'])

Total number of nightlife spots Los Angeles =  138
Total number of nightlife spots New York =  200
Total number of nightlife spots London =  118
Total number of nightlife spots Milan =  99
Total number of nightlife spots Barcelona =  113


Methodology

Display the maps of each city with a marker for each bar

In [6]:
maps[cities[0]]

In [7]:
maps[cities[1]]

In [8]:
maps[cities[2]]

In [9]:
maps[cities[3]]

In [10]:
maps[cities[4]]

Create a dataframe of the bar for each city

In [11]:
df_Los_Angeles=pd.DataFrame.from_dict(df_venues[cities[0]])
df_Los_Angeles.head()

,Name,Address,Lat,Lng
0,Wally's,447 N. Canon Drive,34.071475,-118.401782
1,Silver Lake Wine,2395 Glendale Blvd,34.100003,-118.259315
2,K&L Wine Merchants,1400 Vine St,34.096524,-118.326393
3,Met Her At A Bar,759 S La Brea Ave,34.060557,-118.344744
4,MacLeod Ale Brewing Co.,14741 Calvert St,34.181907,-118.454576


In [12]:
df_New_York=pd.DataFrame.from_dict(df_venues[cities[1]])
df_New_York.head()

,Name,Address,Lat,Lng
0,NR,339 E 75th St,40.770082,-73.954978
1,The Bar Room at Temple Court,123 Nassau St,40.711448,-74.006802
2,Dear Irving,55 Irving Pl,40.736089,-73.987324
3,Attaboy,134 Eldridge St,40.718981,-73.991540
4,Interboro Spirits and Ales,942 Grand St,40.712771,-73.937030


In [13]:
df_London=pd.DataFrame.from_dict(df_venues[cities[2]])
df_London.head()

,Name,Address,Lat,Lng
0,Scarfes Bar,252 High Holborn,51.517813,-0.118184
1,The Foyer & Reading Room,49 Brook St,51.512577,-0.147663
2,The Chesham Arms,15 Mehetabel Rd,51.547727,-0.050131
3,The Beer Shop London,40 Nunhead Green,51.465729,-0.057836
4,The Punch Room,10 Berners St,51.516905,-0.136151


In [14]:
df_Milan=pd.DataFrame.from_dict(df_venues[cities[3]])
df_Milan.head()

,Name,Address,Lat,Lng
0,Bulgari Lounge Bar,Via Privata Fratelli Gabba,45.470014,9.188943
1,Ceresio 7 Pools & Restaurant,"Via Ceresio, 7",45.484025,9.179849
2,La Prosciutteria,"Corso Giuseppe Garibaldi, 53",45.474152,9.183449
3,B Café,Via San Maurilio 20,45.462640,9.183381
4,Bicerin,Via Panfilo Castaldi 24,45.476988,9.202432


In [15]:
df_Barcelona=pd.DataFrame.from_dict(df_venues[cities[4]])
df_Barcelona.head()

,Name,Address,Lat,Lng
0,El Quinto Moño,"Carrer Villaroel, 96",41.384935,2.157213
1,Lo Pinyol,"C. Torrent de L'Olla, 7",41.398598,2.161148
2,Blavis,Saragossa 85,41.403303,2.147298
3,Chivuo's,"C. Pintor Fortuny, 15",41.382961,2.169948
4,Cañete,"C. Unió, 17",41.379154,2.173092


Cluster the bars of each city

In [16]:
from sklearn.cluster import KMeans

In [17]:
k=5
k_cluster=df_Los_Angeles.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Los_Angeles.insert(0,'Cluster Labels',kmeans.labels_)
df_Los_Angeles

,Cluster Labels,Name,Address,Lat,Lng
0,0,Wally's,447 N. Canon Drive,34.071475,-118.401782
1,1,Silver Lake Wine,2395 Glendale Blvd,34.100003,-118.259315
2,0,K&L Wine Merchants,1400 Vine St,34.096524,-118.326393
3,0,Met Her At A Bar,759 S La Brea Ave,34.060557,-118.344744
4,2,MacLeod Ale Brewing Co.,14741 Calvert St,34.181907,-118.454576
5,1,Wurstküche,800 E 3rd St,34.045520,-118.236131
6,0,Terra,10250 Santa Monica Blvd,34.060426,-118.418607
7,0,Das Bunker,4067 W Pico Blvd,34.047810,-118.324116
8,1,Modern Times Beer: The Dankness Dojo,832 S Olive St,34.044133,-118.257600
9,3,accomplice bar,3811 Grand View Blvd,34.004463,-118.431049


In [18]:
k=5
k_cluster=df_New_York.drop(['Address', 'Name'],1).reset_index(drop=True)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_New_York.insert(0,'Cluster Labels',kmeans.labels_)
df_New_York

,Cluster Labels,Name,Address,Lat,Lng
0,2,NR,339 E 75th St,40.770082,-73.954978
1,3,The Bar Room at Temple Court,123 Nassau St,40.711448,-74.006802
2,0,Dear Irving,55 Irving Pl,40.736089,-73.987324
3,0,Attaboy,134 Eldridge St,40.718981,-73.991540
4,4,Interboro Spirits and Ales,942 Grand St,40.712771,-73.937030
5,4,Fifth Hammer Brewing Company,10-28 46th Ave,40.746553,-73.951438
6,2,Bondurants,303 E 85th St,40.777119,-73.952043
7,3,The Raines Law Room,48 W 17th St,40.738745,-73.994483
8,0,Angel’s Share,8 Stuyvesant St,40.729755,-73.989360
9,1,Delta Sky360° Club,4 Penn Plz,40.750564,-73.992824


In [19]:
k=5
k_cluster=df_London.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_London.insert(0,'Cluster Labels',kmeans.labels_)
df_London


,Cluster Labels,Name,Address,Lat,Lng
0,0,Scarfes Bar,252 High Holborn,51.517813,-0.118184
1,0,The Foyer & Reading Room,49 Brook St,51.512577,-0.147663
2,1,The Chesham Arms,15 Mehetabel Rd,51.547727,-0.050131
3,4,The Beer Shop London,40 Nunhead Green,51.465729,-0.057836
4,0,The Punch Room,10 Berners St,51.516905,-0.136151
5,1,Crate Brewery,"Unit 7, White Building",51.542973,-0.022063
6,0,Cahoots,13 Kingly Ct,51.512523,-0.138623
7,1,Howling Hops Tank Bar,Queens Yard,51.542863,-0.022620
8,4,The Mayflower,117 Rotherhithe St,51.501778,-0.053616
9,0,Gordon's Wine Bar,47 Villiers St,51.507911,-0.123293


In [20]:
k=5
k_cluster=df_Milan.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Milan.insert(0,'Cluster Labels',kmeans.labels_)
df_Milan

,Cluster Labels,Name,Address,Lat,Lng
0,4,Bulgari Lounge Bar,Via Privata Fratelli Gabba,45.470014,9.188943
1,4,Ceresio 7 Pools & Restaurant,"Via Ceresio, 7",45.484025,9.179849
2,4,La Prosciutteria,"Corso Giuseppe Garibaldi, 53",45.474152,9.183449
3,1,B Café,Via San Maurilio 20,45.462640,9.183381
4,4,Bicerin,Via Panfilo Castaldi 24,45.476988,9.202432
5,4,Blenderino Burger Bar,NaN,45.480764,9.187217
6,4,BeerShow,Via Borsieri 30,45.489050,9.189068
7,2,Caffe Margherita,Viale Toscana 17,45.446340,9.195253
8,2,El Büscia,Viale Beatrice D'Este 48,45.451835,9.196518
9,4,Niks & Co,Via Copernico,45.490390,9.203749


In [21]:
k=5
k_cluster=df_Barcelona.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Barcelona.insert(0,'Cluster Labels',kmeans.labels_)
df_Barcelona

,Cluster Labels,Name,Address,Lat,Lng
0,1,El Quinto Moño,"Carrer Villaroel, 96",41.384935,2.157213
1,3,Lo Pinyol,"C. Torrent de L'Olla, 7",41.398598,2.161148
2,3,Blavis,Saragossa 85,41.403303,2.147298
3,1,Chivuo's,"C. Pintor Fortuny, 15",41.382961,2.169948
4,1,Cañete,"C. Unió, 17",41.379154,2.173092
5,1,Lambicus Bar,"C. Tamarit, 107",41.375979,2.157355
6,3,Chivuo's,"C. Torrent de l'Olla, 175",41.404940,2.153964
7,2,La Mundana,"Vallespir, 93",41.381136,2.135893
8,1,Bodega Armando,"C. Bisbe Laguarda, 3",41.379974,2.164891
9,4,Flor de Llúpol,"C. Mateu Ferran, 4",41.435005,2.188919


Import libraries

In [22]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Display the maps of each city with the night spots k-clustered

In [23]:
map_colored0=maps[cities[0]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Los_Angeles['Lat'], df_Los_Angeles['Lng'], df_Los_Angeles['Address'], df_Los_Angeles['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored0)
       
map_colored0

In [24]:
map_colored1=maps[cities[1]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_New_York['Lat'], df_New_York['Lng'], df_New_York['Address'], df_New_York['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored1)
       
map_colored1

In [25]:
map_colored2=maps[cities[2]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_London['Lat'], df_London['Lng'], df_London['Address'], df_London['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored2)
       
map_colored2

In [26]:
map_colored3=maps[cities[3]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Milan['Lat'], df_Milan['Lng'], df_Milan['Address'], df_Milan['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored3)
       
map_colored3

In [27]:
map_colored4=maps[cities[4]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Barcelona['Lat'], df_Barcelona['Lng'], df_Barcelona['Address'], df_Barcelona['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored4)
       
map_colored4

Determine how many bar there are in each Los Angeles cluster

In [28]:
counts_LA = df_Los_Angeles['Cluster Labels'].value_counts().to_dict()
df_LA=pd.DataFrame.from_dict(counts_LA, orient='index')
df_LA.columns = ['Bar Number']
df_LA

,Bar Number
1,46
0,34
3,14
2,5
4,1


Determine how many bar there are in each New York cluster

In [29]:
counts_NY = df_New_York['Cluster Labels'].value_counts().to_dict()
df_NY=pd.DataFrame.from_dict(counts_NY, orient='index')
df_NY.columns = ['Bar Number']
df_NY

,Bar Number
0,32
3,30
1,21
2,11
4,6


Determine how many bar there are in each London cluster

In [31]:
counts_Lon = df_London['Cluster Labels'].value_counts().to_dict()
df_Lon=pd.DataFrame.from_dict(counts_Lon, orient='index')
df_Lon.columns = ['Bar Number']
df_Lon


,Bar Number
0,43
1,27
4,18
2,9
3,3


Determine how many bar there are in each Milan cluster

In [32]:
counts_Mil = df_Milan['Cluster Labels'].value_counts().to_dict()
df_Mil=pd.DataFrame.from_dict(counts_Mil, orient='index')
df_Mil.columns = ['Bar Number']
df_Mil

,Bar Number
2,31
4,30
1,22
0,14
3,2


Determine how many bar there are in each Barcelona cluster

In [33]:
counts_Bar = df_Barcelona['Cluster Labels'].value_counts().to_dict()
df_Bar=pd.DataFrame.from_dict(counts_LA, orient='index')
df_Bar.columns = ['Bar Number']
df_Bar

,Bar Number
1,46
0,34
3,14
2,5
4,1
